<a href="https://colab.research.google.com/github/Uday-ashes-uday/Pytorch-with-MNIST-Dataset/blob/main/convolution_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
if torch.cuda.is_available():
  device='cuda'
else:
  device='cpu'

In [3]:
#set hyperparameters

num_epochs=4
batch_size=4
learning_rate=0.001

#transformations to be made

transforms=transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [4]:
#get cifar10 data

train_dataset=torchvision.datasets.CIFAR10(root='/content/cifar/train',
                                           train=True,
                                           download=True,

                                           transform=transforms)

test_dataset=torchvision.datasets.CIFAR10(root='/content/cifar/test',
                                          train=False,
                                          download=True,
                                          transform=transforms)



100%|██████████| 170498071/170498071 [00:02<00:00, 64054996.41it/s]


Extracting /content/cifar/train/cifar-10-python.tar.gz to /content/cifar/train


100%|██████████| 170498071/170498071 [00:01<00:00, 90070938.88it/s]


Extracting /content/cifar/test/cifar-10-python.tar.gz to /content/cifar/test


In [40]:
#convert datasets into dataloaders

train_loader=torch.utils.data.DataLoader(train_dataset,
                                         batch_size=32,
                                         shuffle=True)

test_loader=torch.utils.data.DataLoader(test_dataset,
                                        batch_size=32,
                                        shuffle=False)

In [41]:
#class labels

labels=['plane','car','bird','cat','deer','dog','frog','horse','ship','truck']

In [42]:
#implement convnet

class ConvNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1=nn.Conv2d(3,6,5)
    self.pool=nn.MaxPool2d(2,2)
    self.conv2=nn.Conv2d(6,16,5)
    self.fc1=nn.Linear(16*5*5,120)
    self.fc2=nn.Linear(120,84)
    self.fc3=nn.Linear(84,10)

  def forward(self,x):
    x=self.pool(F.relu(self.conv1(x)))
    #print(x.shape)
    x=self.pool(F.relu(self.conv2(x)))
    #print(x.shape)
    x=x.view(-1,16*5*5)
    x=F.relu(self.fc1(x))
    x=F.relu(self.fc2(x))
    x=self.fc3(x)
    return x







In [43]:
#instantiate model

model=ConvNet().to(device)

#set loss function and an optimizer

loss_fn=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [44]:
len(train_loader)

1563

In [58]:
num_epochs=30
n_total_steps=len(train_loader)

for epoch in range(num_epochs):

  for i,(images,labels)in enumerate(train_loader):

    images=images.to(device)
    labels=labels.to(device)


    #forward pass

    outs=model(images)
    loss=loss_fn(outs,labels)

    #backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1)%200 ==0:
      print(f'Epoch {epoch+1}/{num_epochs}  Step{i+1}/{n_total_steps} Loss {loss.item():.4f}')




Epoch 1/30  Step200/1563 Loss 2.1371
Epoch 1/30  Step400/1563 Loss 1.8735
Epoch 1/30  Step600/1563 Loss 1.6735
Epoch 1/30  Step800/1563 Loss 1.8331
Epoch 1/30  Step1000/1563 Loss 1.6128
Epoch 1/30  Step1200/1563 Loss 1.8332
Epoch 1/30  Step1400/1563 Loss 1.7636
Epoch 2/30  Step200/1563 Loss 1.7374
Epoch 2/30  Step400/1563 Loss 1.5659
Epoch 2/30  Step600/1563 Loss 1.6621
Epoch 2/30  Step800/1563 Loss 1.6839
Epoch 2/30  Step1000/1563 Loss 1.8641
Epoch 2/30  Step1200/1563 Loss 1.7745
Epoch 2/30  Step1400/1563 Loss 1.7378
Epoch 3/30  Step200/1563 Loss 1.8354
Epoch 3/30  Step400/1563 Loss 1.6811
Epoch 3/30  Step600/1563 Loss 1.8255
Epoch 3/30  Step800/1563 Loss 1.4254
Epoch 3/30  Step1000/1563 Loss 1.7093
Epoch 3/30  Step1200/1563 Loss 1.8352
Epoch 3/30  Step1400/1563 Loss 1.8435
Epoch 4/30  Step200/1563 Loss 1.5651
Epoch 4/30  Step400/1563 Loss 1.9120
Epoch 4/30  Step600/1563 Loss 1.5760
Epoch 4/30  Step800/1563 Loss 1.7096
Epoch 4/30  Step1000/1563 Loss 1.6262
Epoch 4/30  Step1200/1563 Lo

In [62]:
with torch.no_grad():
  n_correct=0
  n_samples=0
  for images,labels in train_loader:
    images=images.to(device)
    labels=labels.to(device)
    outs=model(images)
    _,predicted=torch.max(outs,1)
    n_samples+=len(images)
    n_correct+=(predicted==labels).sum().item()
  acc=(n_correct/n_samples)*100
  print(f'accuracy of this network {acc:.2f}%')


accuracy of this network 55.38%
